# Replication of COCKATIEL

## Prepare RoBERTa Model

RoBERTa model with **a non-negative (ReLU)** last layer to discover concepts using COCKATIEL.
We use the `features` method to compute the activations on which to apply the NMF, `end_model` to go
from these activations to the final prediction.

In [2]:
import torch
from torch import nn
from torch.nn import MSELoss
from transformers import RobertaPreTrainedModel, RobertaModel

In [ ]:
# A custom fully-connected classification head for RoBERTa with a non-negative layer on which we can compute the NMF.

class CustomRobertaClassificationHead(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, 2)

    def forward(self, features):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.out_proj(x) # Projection layer, yani multi-head attention katmanının yüksek boyutlu output'unu düşürmek için project yapar
                             # Ardından final sınıflandırma logits'leri hesaplanır
        return x

    # out_proj olmadan feature'ları hesaplayan fonk.
    def features(self, x):
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.relu(x)
        return x

    # forward()'ın sonunda yapılan işlemin aynısı, ayrı fonk. olarak da tanımlamışlar
    def end_model(self, x):
        x = self.dropout(x)
        x = self.out_proj(x)

        return x

In [ ]:
# A custom RoBERTa model using a custom fully-connected head with a non-negative layer on which we can compute the NMF.
    
class CustomRobertaForSequenceClassification(RobertaPreTrainedModel):

    # Model yüklenirken position_ids'i ignore etsin
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        
        # Regresyon problemi gibi ele aldıklarından 1 demişler, yoksa sentiment a. için 2 olmalıydı
        self.num_labels = 1
        self.config = config

        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.classifier = CustomRobertaClassificationHead(config)
        
        # Yine regresyon olmasından dolayı
        self.mse_loss = MSELoss()

        # Modelin başka hiperparametrelerini initialize etmek için ek fonk.
        self.post_init()

    def features(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        features = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        return self.classifier.features(features[0][:, 0, :])

    def end_model(self, activations):
        return self.classifier.end_model(activations)

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0][:, 0, :]
        logits = self.classifier(sequence_output)

        return logits

## COCKATIEL: Explainability Component

In [ ]:
import numpy as np
import torch
from sklearn.decomposition import NMF
from math import ceil

from .sampling import ScipySobolSequence, concept_perturbation
from .sobol import JansenEstimator
from .utils import tokenize

from typing import Callable, List

In [ ]:
class COCKATIEL:
    """
    model
        The Torch hugging-face model that we wish to explain. It MUST have a non-negative layer on
        which to extract the concepts.
    tokenizer
        A callable object to transform strings into inputs for the model.
    components
        An integer for the amount of concepts we wish to discover in the activation space.
    batch_size
        The batch size for all the operations that use the model
    device
        The type of device on which to place the torch tensors
    """
    sobol_nb_design = 32

    def __init__(
            self,
            model,
            tokenizer: Callable,
            components: int = 25,
            batch_size: int = 256,
            device: str = 'cuda',
            nmf_max_iter: int = 1000
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.components = components
        self.batch_size = batch_size
        self.device = device
        self.max_iter = nmf_max_iter

    def extract_concepts(
            self,
            cropped_dataset: List[str],
            dataset: List[str],
            class_id: int,
            limit_sobol: int = 1000
    ):
        """
        Extracts the concepts following the object's parameters.

        Parameters
        ----------
        cropped_dataset
            The dataset containing the excerpts used to discover the concepts.
        dataset
            A sample of the dataset (with whole inputs) on which to compute the Sobol importance.
        class_id
            An integer for the class we wish to explain.
        limit_sobol
            The maximum amount of masks to use for estimating Sobol indices.

        Returns
        -------
        excerpts
            The excerpts used to learn the concepts.
        u_excerpts
            The coefficients in the learned concept base for the excerpts.
        factorization
            The object to transform activations using the concept base.
        global_importance
            An array with the global importance of each concept (Sobol indices).
        """
        excerpts = []
        excerpts_activations = None

        with torch.no_grad():
            for batch_id in range(ceil(len(cropped_dataset) / self.batch_size)):
                batch_start = batch_id * self.batch_size
                batch_end = batch_start + self.batch_size

                batch_sentences = cropped_dataset[batch_start:batch_end]
                batch_tokenized = tokenize(batch_sentences, self.tokenizer, self.device)

                batch_activations = self.model.features(**batch_tokenized)

                excerpts_activations = batch_activations if excerpts_activations is None \
                    else torch.cat([excerpts_activations, batch_activations], 0)
                excerpts = batch_sentences if excerpts is None \
                    else excerpts + batch_sentences

        points_activations = None
        with torch.no_grad():
            for batch_id in range(ceil(len(dataset) / self.batch_size)):
                batch_start = batch_id * self.batch_size
                batch_end = batch_start + self.batch_size
                tokenized_batch = tokenize(dataset[batch_start:batch_end], self.tokenizer, self.device)
                activations = self.model.features(**tokenized_batch)
                points_activations = activations if points_activations is None else torch.cat(
                    [points_activations, activations])

        # applying GAP(.) on the activation and ensure positivity if needed
        excerpts_activations = self._preprocess(excerpts_activations)
        points_activations = self._preprocess(points_activations)

        # using the activations, we will now use the matrix factorization to
        # find the concept bank (W) and the concept representation (U) of the
        # segments and the points
        factorization = NMF(n_components=self.components, max_iter=self.max_iter)

        u_excerpts = factorization.fit_transform(excerpts_activations)
        W = torch.Tensor(factorization.components_).float().to(self.device)

        # we don't need segments activations anymore, the concept bank is trained
        del excerpts_activations

        # using the concept bank and the points, we will now evaluate the importance of
        # each concept for each points to get a global importance score for each
        # concept in the concept bank
        global_importance = self._sobol_importance(cropped_dataset, points_activations[:limit_sobol], class_id, W)

        return excerpts, u_excerpts, factorization, global_importance

    def _sobol_importance(self, cropped_dataset, activations: torch.Tensor, class_id: int, W: torch.Tensor):
        """
        Computes the Sobol indices using the dataset containing the excerpts and the activations from the
        dataset (whole inputs) for the target class, and for a fixed (already learned) concept base W.

        Parameters
        ----------
        cropped_dataset
            The activations of the dataset containing the excerpts used to discover the concepts.
        activations
            The activations for inputs from the original dataset.
        class_id
            An integer for the class we wish to explain.
        W
            The (already learned) concept base.

        Returns
        -------
        global_importance
            An array with the Sobol indices
        """
        masks = ScipySobolSequence()(self.components, nb_design=self.sobol_nb_design)
        estimator = JansenEstimator()

        if not isinstance(W, torch.Tensor):
            W = torch.Tensor(W).float().to(self.device)

        importances = []
        for act in activations:
            act = torch.Tensor(act).float().to(self.device)

            y_pred = None
            for batch_id in range(ceil(len(cropped_dataset) / self.batch_size)):
                batch_start = batch_id * self.batch_size
                batch_end = batch_start + self.batch_size
                batch_masks = torch.Tensor(masks[batch_start:batch_end]).float().to(self.device)

                y_batch = concept_perturbation(self.model, act, batch_masks, class_id, W)
                y_pred = y_batch if y_pred is None else torch.cat([y_pred, y_batch], 0)

            if self.device == 'cuda' or self.device == torch.device('cuda'):
                y_pred = y_pred.cpu()
            stis = estimator(masks, y_pred.numpy(), self.sobol_nb_design)
            importances.append(stis)

        global_importance = np.mean(importances, 0)

        return global_importance

    def _preprocess(self, activations: torch.Tensor):
        """
        Preprocesses the activations to make sure that they're the right shape for being input to the
        NMF algorithm later.

        Parameters
        ----------
        activations
            The (non-negative) activations from the model under study.

        Returns
        -------
        activations
            The preprocessed activations, ready for COCKATIEL.
        """
        if len(activations.shape) == 4:
            activations = torch.mean(activations, (1, 2))

        if torch.min(activations) < 0:
            raise ValueError("Please choose a layer with positive activations.")
        if self.device == 'cuda' or self.device == torch.device('cuda'):
            activations = activations.cpu()

        return activations.numpy().astype(np.float32)

## Occlusion: Local Part

In [ ]:
import numpy as np
import nltk
import sklearn.decomposition
from nltk.tokenize import word_tokenize

from .utils import extract_clauses, calculate_u_values, calculate_importance

from typing import List, Callable, Optional, Union, Tuple

nltk.download('punkt')

In [ ]:
"""
This module implements the local part of COCKATIEL: occlusion. This allows us to estimate the presence of
concepts in parts of the input text.
"""

def occlusion_concepts(
        sentence: str,
        model,
        tokenizer: Callable,
        factorization: Union[sklearn.decomposition.NMF, Tuple[sklearn.decomposition.NMF, sklearn.decomposition.NMF]],
        l_concept_id: Union[np.ndarray, Tuple[np.ndarray, np.ndarray]],
        ignore_words: Optional[List[str]] = None,
        two_labels: bool = True,
        extract_fct: str = "clause",
        device='cuda'
) -> np.ndarray:
    """
    Generates explanations for the input sentence using COCKATIEL.

    If two_labels is False, it computes the presence of the concepts of interest (in l_concept_id) using the
    NMF object in factorization.
    If two_labels is True, it computes the presence of the concepts of interest in the tuple of l_concept_id using
    the tuple of NMF objects in factorization (to do so for both classes in imdb-reviews task).

    The granularity of the explanations is set with extract_fct.

    Parameters
    ----------
    sentence
        The string (sentence) we wish to explain using COCKATIEL.
    model
        The model under study.
    tokenizer
        A Callable that transforms strings into tokens capable of being ingested by the model.
    factorization
        Either the NMF object to transform activations into the concept base, or a tuple with an object for each of
        the two classes (for the imdb-reviews task) if two_labels is True.
    l_concept_id
        Either a list of concepts of interest (for a given task) or a tuple with two lists, one for each class (if
        two_labels is True).
    ignore_words
        A list of strings to ignore when applying occlusion.
    two_labels
        A bool indicating whether we wish to explain only one class or both (for imdb-reviews task).
    extract_fct
        A string indicating whether at which level we wish to explain: "word", "clause" or "sentence".
    device
        The device on which tensors are stored ("cpu" or "cuda").

    Returns
    -------
    l_importances
        An array with the presence of each concept in the input sentence.
    """
    sentence = str(sentence)

    if extract_fct == "clause":
        words = extract_clauses(sentence, clause_type=None)
        separate = " "

    else:
        words = word_tokenize(sentence)
        if extract_fct == "sentence":
            separate = ". "
        elif extract_fct == "word":
            separate = " "
        else:
            raise ValueError("Extraction function can be only 'clause', 'sentence', or 'word")

    if two_labels:
        u_values_pos = calculate_u_values(sentence, words,  model, tokenizer, factorization[0], separate, ignore_words, device)
        u_values_neg = calculate_u_values(sentence, words,  model, tokenizer, factorization[1], separate, ignore_words, device)
        l_importances = []
        for concept_id in l_concept_id[0]:
            importances = calculate_importance(words, u_values_pos, concept_id, ignore_words)
            l_importances.append(np.array(importances))
        for concept_id in l_concept_id[1]:
            importances = calculate_importance(words, u_values_neg, concept_id, ignore_words)
            l_importances.append(np.array(importances))

    else:  # look at only one class:
        u_values = calculate_u_values(sentence, words,  model, tokenizer, factorization, separate, ignore_words, device)
        l_importances = []
        for concept_id in l_concept_id:
            importances = calculate_importance(words, u_values, concept_id, ignore_words)
            l_importances.append(np.array(importances))

    return np.array(l_importances)


## Sampling

In [ ]:
from abc import ABC, abstractmethod
import numpy as np
import scipy

In [ ]:
"""
Sampling methods for replicated designs
"""

class Sampler(ABC):
    """
    Base class for replicated design sampling.
    """

    @staticmethod
    def build_replicated_design(sampling_a, sampling_b):
        """
        Build the replicated design matrix C using A & B

        Parameters
        ----------
        sampling_a
          The masks values for the sampling matrix A.
        sampling_b
          The masks values for the sampling matrix B.

        Returns
        -------
        replication_c
          The new replicated design matrix C generated from A & B.
        """
        replication_c = np.array([sampling_a.copy() for _ in range(sampling_a.shape[-1])])
        for i in range(len(replication_c)):
            replication_c[i, :, i] = sampling_b[:, i]

        replication_c = replication_c.reshape((-1, sampling_a.shape[-1]))

        return replication_c

    @abstractmethod
    def __call__(self, dimension, nb_design):
        raise NotImplementedError()


class ScipySampler(Sampler):
    """
    Base class based on Scipy qmc module for replicated design sampling.
    """

    def __init__(self):
        try:
            self.qmc = scipy.stats.qmc
        except AttributeError as err:
            raise ModuleNotFoundError("COCKATIEL need scipy>=1.7 to use this sampling.") from err


class ScipySobolSequence(ScipySampler):
    """
    Scipy Sobol LP tau sequence sampler.

    Ref. I. M. Sobol., The distribution of points in a cube and the accurate evaluation of
    integrals (1967).
    https://www.sciencedirect.com/science/article/abs/pii/0041555367901449
    """

    def __call__(self, dimension, nb_design):
        sampler = self.qmc.Sobol(dimension * 2, scramble=False)
        sampling_ab = sampler.random(nb_design).astype(np.float32)
        sampling_a, sampling_b = sampling_ab[:, :dimension], sampling_ab[:, dimension:]
        replicated_c = self.build_replicated_design(sampling_a, sampling_b)

        return np.concatenate([sampling_a, sampling_b, replicated_c], 0)


def concept_perturbation(model, activation, masks, class_id, W):
    """
    Apply perturbation on the concept before reconstruction and get the perturbated outputs.
    For NMF we recall that A = U @ W

    Parameters
    ----------
    model
      Model that map the concept layer to the output (h_l->k in the paper)
    activation
      Specific activation to apply perturbation on.
    masks
      Arrays of masks, each of them being a concept perturbation.
    class_id
      Id the class to test.
    W
      Concept bank extracted using NMF.

    Returns
    -------
    y
      Outputs of the perturbated points.
    """
    perturbation = masks @ W

    if len(activation.shape) == 3:
        perturbation = perturbation[:, None, None, :]

    activation = activation[None, :]
    perturbated_activations = activation + perturbation * activation
    y = model.end_model(perturbated_activations)[:, class_id]

    return y

## Total Sobol Indices

In [ ]:
from abc import ABC, abstractmethod
import numpy as np

In [ ]:
"""
Sobol' total order estimators module
"""

class SobolEstimator(ABC):
    """
    Base class for Sobol' total order estimators.
    """

    @staticmethod
    def masks_dim(masks):
        """
        Deduce the number of dimensions using the sampling masks.

        Parameters
        ----------
        masks
          Low resolution masks (before upsampling) used, one for each output.

        Returns
        -------
        nb_dim
          The number of dimensions under study according to the masks.
        """
        nb_dim = np.prod(masks.shape[1:])
        return nb_dim

    @staticmethod
    def split_abc(outputs, nb_design, nb_dim):
        """
        Split the outputs values into the 3 sampling matrices A, B and C.

        Parameters
        ----------
        outputs
          Model outputs for each sample point of matrices A, B and C (in order).
        nb_design
          Number of points for matrices A (the same as B).
        nb_dim
          Number of dimensions to estimate.

        Returns
        -------
        a
          The results for the sample points in matrix A.
        b
          The results for the sample points in matrix B.
        c
          The results for the sample points in matrix C.
        """
        sampling_a = outputs[:nb_design]
        sampling_b = outputs[nb_design:nb_design*2]
        replication_c = np.array([outputs[nb_design*2 + nb_design*i:nb_design*2 + nb_design*(i+1)]
                      for i in range(nb_dim)])
        return sampling_a, sampling_b, replication_c

    @staticmethod
    def post_process(stis, masks):
        """
        Post processing ops on the indices before sending them back. Makes sure the data
        format and shape is correct.

        Parameters
        ----------
        stis
          Total order Sobol' indices, one for each dimensions.
        masks
            Low resolution masks (before upsampling) used, one for each output.

        Returns
        -------
        stis
          Total order Sobol' indices after post processing.
        """
        stis = np.array(stis, np.float32)
        return stis.reshape(masks.shape[1:])

    @abstractmethod
    def __call__(self, masks, outputs, nb_design):
        """
        Compute the Sobol' total order indices according to the Jansen algorithm.

        Ref. Jansen, M., Analysis of variance designs for model output (1999)
        https://www.sciencedirect.com/science/article/abs/pii/S0010465598001544

        Parameters
        ----------
        masks
          Low resolution masks (before upsampling) used, one for each output.
        outputs
          Model outputs associated to each masks. One for each sample point of
          matrices A, B and C (in order).
        nb_design
          Number of points for matrices A (the same as B).

        Returns
        -------
        sti: ndarray
          Total order Sobol' indices, one for each dimensions.
        """
        raise NotImplementedError()


class JansenEstimator(SobolEstimator):
    """
    Jansen estimator for total order Sobol' indices.

    Ref. Jansen, M., Analysis of variance designs for model output (1999)
    https://www.sciencedirect.com/science/article/abs/pii/S0010465598001544
    """

    def __call__(self, masks, outputs, nb_design):
        """
        Compute the Sobol' total order indices according to the Jansen algorithm.

        Parameters
        ----------
        masks
          Low resolution masks (before upsampling) used, one for each output.
        outputs
          Model outputs associated to each masks. One for each sample point of
          matrices A, B and C (in order).
        nb_design
          Number of points for matrices A (the same as B).

        Returns
        -------
        sti
          Total order Sobol' indices, one for each dimensions.
        """
        nb_dim = self.masks_dim(masks)
        sampling_a, _, replication_c = self.split_abc(outputs, nb_design, nb_dim)

        mu_a = np.mean(sampling_a)
        var = np.sum([(v - mu_a)**2 for v in sampling_a]) / (len(sampling_a) - 1)

        stis = [
            np.sum((sampling_a - replication_c[i])**2.0) / (2 * nb_design * var)
            for i in range(nb_dim)
        ]

        return self.post_process(stis, masks)
    

## Util Functions

In [ ]:
import torch
from math import ceil
import numpy as np

from flair.models import SequenceTagger
from flair.data import Sentence

from typing import List, Callable, Union, Optional, Tuple

In [ ]:
# load tagger
tagger = SequenceTagger.load("flair/chunk-english")

In [ ]:
def batcher(elements, batch_size: int):
    """
    An iterable to create batches from a list of elements
    """
    nb_batchs = ceil(len(elements) / batch_size)

    for batch_i in range(nb_batchs):
        batch_start = batch_i * batch_size
        batch_end = batch_start + batch_size

        batch = elements[batch_start:batch_end]
        yield batch

In [ ]:
def tokenize(samples: List[str], tokenizer: Callable, device='cuda'):
    """
    A function to transform a list of strings into tokens to be consumed by the transformer model.
    """
    x = tokenizer(
        [s for s in samples],
        padding="max_length",
        max_length=512,
        truncation=True,
        return_tensors='pt'
    ).to(device)

    return x

In [ ]:
def preprocess(samples: List[Tuple[str, str]], tokenizer: Callable, device='cuda'):
    """
    A basic pre-processing function to transform the format from the imdb dataset to
    something easier to work with.
    """
    x, y = samples[:, 0], samples[:, 1]
    x = tokenize(x, tokenizer, device)
    y = torch.Tensor(y == 'positive').int().to(device)
    return x, y

In [ ]:
def batch_predict(model, tokenizer: Callable, inputs: List[Tuple[str, str]], batch_size: int = 64, device='cuda'):
    """
    A function to pre-process and predict using the transformer model in batches.
    """
    predictions = None
    labels = None

    with torch.no_grad():
        for batch_input in batcher(inputs, batch_size):
            xp, yp = preprocess(batch_input, tokenizer, device)
            out_batch = model(**xp)
            predictions = out_batch if predictions is None else torch.cat([predictions, out_batch])
            labels = yp if labels is None else torch.cat([labels, yp])

        return predictions, labels

In [ ]:
def extract_clauses(ds_entry: Union[List[str], str], clause_type=['NP', 'ADJP']) -> List[str]:
    """
    Separates the input texts into clauses, and only keeps the ones belonging to the specified types.
    If clause_type is None, the texts are split but all the clauses are kept.

    Parameters
    ----------
    ds_entry
        A list of strings that we wish to separate into clauses.
    clause_type
        A list with the types of clauses to keep. If None, all clauses are kept.

    Returns
    -------
    clause_list
        A list with input texts split into clauses.
    """
    s = Sentence(ds_entry)
    tagger.predict(s)
    clause_list = []
    for segment in s.get_labels():
        if clause_type is None:
            clause_list.append(segment.data_point.text)
        elif segment.value in clause_type:
            clause_list.append(segment.data_point.text)

    return clause_list

In [ ]:
def batch_activations_fct(model, inputs: List[str], batch_size=64) -> torch.Tensor:
    """
    A function to extract the activations of input texts in batches.
    """
    activations = None
    with torch.no_grad():
        for batch_input in batcher(inputs, batch_size):
            out_batch = model.features(**batch_input)
            activations = out_batch if activations is None else torch.cat([activations, out_batch])
        return activations

In [ ]:
def acti_preprocess(activations: torch.Tensor) -> np.ndarray:
    """
    A function to preprocess the activations to work with COCKATIEL
    """
    if len(activations.shape) == 4:
        activations = torch.mean(activations, (1, 2))

    if isinstance(activations, np.ndarray):
        activations = torch.Tensor(activations)
    if torch.min(activations) < 0:
        raise ValueError("Please choose a layer with positive activations.")

    return activations.cpu().numpy().astype(np.float32)

In [ ]:
def calculate_u_values(sentence, cropped_sentences, model, tokenizer, factorization,
                       separate, ignore_words: Optional[List[str]] = None, device='cuda') -> np.ndarray:
    if ignore_words is None:
        ignore_words = []
    with torch.no_grad():
        activations = None
        for crop_id in range(-1, len(cropped_sentences)):
            if crop_id == -1:
                perturbated_review = sentence
            elif cropped_sentences[crop_id] not in ignore_words:
                perturbated_review = separate.join(np.delete(cropped_sentences, crop_id))
            else:
                continue
            tokenized_perturbated_review = tokenizer(perturbated_review, truncation=True, padding=True, return_tensors="pt").to(device)
            activation = model.features(**tokenized_perturbated_review)
            activations = activation if activations is None else torch.cat([activations, activation])

        activations = acti_preprocess(activations)
        u_values = factorization.transform(activations)
        
        return u_values

In [ ]:
def calculate_importance(
        words: List[str], u_values: np.ndarray, concept_id: int, ignore_words: List[str]
) -> List[float]:
    """
    Calculates the presence of concepts in the input list of words.
    """
    u_delta = u_values[0, concept_id] - u_values[1:, concept_id]
    importances = []
    delta_id = 0  # pointer to get current id in importance (as we skip unused word)

    for word_id in range(len(words)):
        if words[word_id] not in ignore_words:
            importances.append(u_delta[delta_id])
            delta_id += 1
        else:
            importances.append(0.0)

    return importances

## Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import nltk
from nltk.tokenize import word_tokenize

from .utils import extract_clauses

from typing import List, Optional

nltk.download('punkt')
plt.style.use('seaborn')

In [ ]:
"""
A module for generating plots for the explanations on notebooks.
"""

def print_legend(colors, label_to_criterion):
    """
    Prints the legend for the plot in different colors.

    Parameters
    ----------
    colors
        A dictionary with the colors for each label.
    label_to_criterion
        A dictionary with the text to put on each label.
    """
    html = []
    for label_id in label_to_criterion.keys():
        html.append(f'<span style="background-color: {colors[label_id]} 0.5); padding: 1px 5px; border: solid 3px ; border-color: {colors[label_id]} 1); #EFEFEF">{label_to_criterion[label_id]} </span>')
    display(HTML("<div style='display: flex; width: 400px; flex-wrap: wrap'>" +  " ".join(html) + " </div>" ))
    display(HTML('<br><br>'))


def viz_concepts(
        text,
        explanation,
        colors,
        ignore_words: Optional[List[str]] = None,
        extract_fct: str = "clause"
):
    """
    Generates the visualization for COCKATIEL's explanations.

    Parameters
    ----------
    text
        A string with the text we wish to explain.
    explanation
        An array that corresponds to the output of the occlusion function.
    ignore_words
        A list of strings to ignore when applying occlusion.
    extract_fct
        A string indicating whether at which level we wish to explain: "word", "clause" or "sentence".
    colors
        A dictionary with the colors for each label
    """
    try:
        text = text.decode('utf-8')
    except:
        text = str(text)

    if extract_fct == "clause":
        words = extract_clauses(text, clause_type=None)
    else:
        words = word_tokenize(text)

    l_phi = np.array(explanation)

    phi_html = []

    p = 0  # pointer to get current color for the words (it does not color words that have no phi)
    for i in range(len(words)):
        if words[i] not in ignore_words:
            k = 0
            for j in range(len(l_phi)):
                if l_phi[k][p] < l_phi[j][p]:
                    k = j

            if l_phi[k][p] > 0.2:
                phi_html.append(f'<span style="background-color: {colors[k]} {l_phi[k][p]}); padding: 1px 5px; border: solid 3px ; border-color: {colors[k]} 1); #EFEFEF">{words[i]}</span>')
                p += 1
            else:
                phi_html.append(f'<span style="background-color: rgba(233,30,99,0);  padding: 1px 5px; border: solid 3px ; border-color:  rgba(233,30,99,0); #EFEFEF">{words[i]}</span>')
                p += 1
        else:
            phi_html.append(f'<span style="background-color: rgba(233,30,99,0);  padding: 1px 5px; border: solid 3px ; border-color:  rgba(233,30,99,0); #EFEFEF">{words[i]}</span>')
    display(HTML("<div style='display: flex; width: 400px; flex-wrap: wrap'>" +  " ".join(phi_html) + " </div>" ))
    display(HTML('<br><br>'))

## Run